In [1]:
from tqdm import tqdm
import json
import numpy as np
import copy

import spot_utils
import synthTL

In [2]:
translate_dict = synthTL.create_translate_dict('metadata/rawcontext_decomposition-worker.xlsx')
cur_nl_spec_fname = "specs/amba_worker.txt"
test_str = open(cur_nl_spec_fname,"r").read()
formula_DUT = translate_dict[test_str]
cur_graph = synthTL.Node(test_str)
synthTL.cur_DUT_variables = spot_utils.get_variables(formula_DUT)

In [3]:
exp_name = "metadata/ambaworker_llm-3"

In [4]:
def load_llm_translation(exp_name):
    cur_retranslation_dict = json.load(open(exp_name+"_translations.json"))
    cur_redecomposition_dict = json.load(open(exp_name+"_decompositions.json"))
    all_retranslation_dict = json.load(open(exp_name+"_alltranslations.json"))
    all_redecomposition_dict = json.load(open(exp_name+"_alldecompositions.json"))
    correct_abs_id_list = json.load(open(exp_name+"_correctednodeids.json"))
    return cur_retranslation_dict, cur_redecomposition_dict, all_retranslation_dict, all_redecomposition_dict, correct_abs_id_list

In [5]:
cur_test_list = [synthTL.Node(test_str)]

In [6]:
def set_translation_ambiguity(retranslation_dict,redecomposition_dict,all_retranslation_dict,correct_abs_id_list,k_ambiguity = 1,total_ambiguity_threshold = 2**14):
    np.random.seed(0)
    order_idx = np.random.choice(len(correct_abs_id_list),size=len(correct_abs_id_list),replace=False)
    cur_retranslation_dict = copy.deepcopy(retranslation_dict)
    cur_ambiguity = synthTL.count_graphs(cur_test_list[0],cur_retranslation_dict,redecomposition_dict,mode='baseline')
    for next_id_query in [correct_abs_id_list[idx] for idx in order_idx]:
        if len(cur_retranslation_dict[next_id_query]) < k_ambiguity and cur_ambiguity*k_ambiguity < total_ambiguity_threshold:
            cur_translation_list = synthTL.remove_duplicate_translations(all_retranslation_dict[next_id_query])
            extra_select = cur_translation_list[:k_ambiguity-1]
            cur_retranslation_dict[next_id_query] = retranslation_dict[next_id_query] + extra_select
            cur_ambiguity = synthTL.count_graphs(cur_test_list[0],cur_retranslation_dict,redecomposition_dict,mode='baseline')
    print(cur_ambiguity)
    return cur_retranslation_dict
    
cur_retranslation_dict, cur_redecomposition_dict, all_retranslation_dict, all_redecomposition_dict, correct_abs_id_list = load_llm_translation(exp_name)
cur_retranslation_dict = set_translation_ambiguity(
                            cur_retranslation_dict,
                            cur_redecomposition_dict,
                            all_retranslation_dict,
                            correct_abs_id_list,
                            k_ambiguity = 2,
                            total_ambiguity_threshold = 2**13)

4096


In [7]:
%%time
checked_graphs = set()
mc_graph_list = synthTL.constrain_decomposition_for_node(cur_test_list[0],
                                               node=cur_test_list[0],
                                               retranslation_dict=cur_retranslation_dict,
                                               redecomposition_dict=cur_redecomposition_dict,
                                               #redecomposition_dict=graph_to_decompose_cache(mc_holds_graph_list[0]),
                                               formula_DUT=formula_DUT,
                                               dec_cache={},
                                               visited_set=set(),
                                               checked_graphs=checked_graphs)

CPU times: user 1min 32s, sys: 427 ms, total: 1min 32s
Wall time: 1min 32s


In [8]:
len(mc_graph_list)

3

In [9]:
any([spot_utils.check_equivalent(g.translation,formula_DUT) for g in mc_graph_list])

True

In [10]:
filtered_list = synthTL.filter_graph_list(graph_list=mc_graph_list,filter_func=synthTL.get_most_constrained)

In [11]:
assert spot_utils.check_equivalent(filtered_list[0].translation,formula_DUT)

In [12]:
len(synthTL.get_all_descendants(filtered_list[0]))

48

In [13]:
synthTL.count_graphs(cur_test_list[0],cur_retranslation_dict,cur_redecomposition_dict,mode='baseline')

4096

In [14]:
len(checked_graphs)

316